In [3]:
from landslide4sense.data import LandslideDataSet
import albumentations as A

ImportError: cannot import name '_registerMatType' from 'cv2.cv2' (/usr/local/lib/python3.7/dist-packages/cv2/cv2.cpython-37m-x86_64-linux-gnu.so)

In [4]:
A

NameError: name 'A' is not defined